In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import imutils
from imutils.video import VideoStream

In [2]:
def detect_and_predict_mask(frame,faceNet,maskNet):
    # Grab the Dimension of the frame and then construct the a blob
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    # Initialize Our List of Faces,their corresponding Locations and List of prediction
    
    faces = []
    locs = []
    preds = []
    
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence>0.5:
            # We Need the X and Y coordinates
            box =detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY) = box.astype('int')
            
            # We Need to ensure the bounding boxes within the dimension of the frame
            (startX,startY) = (max(0,startX),max(0,startY))
            (endX,endY) = (min(w-1,endX),min(h-1,endY))
        
            # Extract the Face RoI, convert it from BGR to RGB Channel,resize it to 224,224
            # and Preprocess it
            face = frame[startY:endY,startX:endX]
            face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face = cv2.resize(face,(224,224))
            face = img_to_array(face)
            face = preprocess_input(face)
        
        
            faces.append(face)                   
            locs.append((startX,startY,endX,endY))
                
            #Only make a Predictions if atleast one face was detected
    
        if len(face)>0:
            faces = np.array(faces,dtype = 'float32')
            preds = maskNet.predict(faces,batch_size=12)
        
        return (locs,preds)
        

In [3]:
prototxtPath = os.path.sep.join([r'D:\Deep Learning Project\Mask Detection Model Deep Learning','deploy.prototxt'])
weightsPath =  os.path.sep.join([r'D:\Deep Learning Project\Mask Detection Model Deep Learning','res10_300x300_ssd_iter_140000.caffemodel'])


In [4]:
faceNet = cv2.dnn.readNet(prototxtPath,weightsPath)

In [5]:
# Model Load
maskNet = load_model(r'D:\Deep Learning Project\Mask Detection Model Deep Learning\mobile_netv2.model')

In [9]:
vs = VideoStream(src=0).start()

while True:
    frame = vs.read()
    frame = imutils.resize(frame,width=400)
    
    #Detect Faces in the frame and predict if they are wearing Mask or not
    (locs,preds) = detect_and_predict_mask(frame,faceNet,maskNet)
    
    # Loop Over the Detected Face locations and their corresponding locations
    for (box,pred) in zip(locs,preds):
         (startX,startY,endX,endY) = box
         (mask,withoutmask)  = pred
         #Determine the class label and color we will use to draw the bounding box and text
         label = 'Mask' if mask>withoutmask else 'No Mask'
         color = (0,255,0) if label == 'Mask' else (0,0,255)
        
         # Display the label and Bounding boxes
        
         cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
         cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
        
    # Show the output frame
    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cv2.destroyAllWindows()
vs.stop()


UnboundLocalError: local variable 'face' referenced before assignment